Pour cet exemple, nous appliquons l'attaque Groundhog de Stadler et al. au fichier d'enseignement du recensement 1% du recensement d'Angleterre et du Pays de Galles et au générateur Raw le plus simple.

L'objectif de TAPAS est d'évaluer la possibilité d'une attaque, plutôt que de former et de déployer des attaques contre des ensembles de données réels. Cela éclaire les décisions de conception prises, notamment en ce qui concerne les connaissances auxiliaires.

Cet exemple se veut une introduction générale à TAPAS et explique certains choix de conception importants à faire lors de l'utilisation de la boîte à outils.

Nous importons d’abord les dépendances nécessaires pour exécuter notre attaque (dans ce cas, les différents modules TAPAS).

In [3]:
import tapas.datasets
import tapas.generators
import tapas.threat_models
import tapas.attacks
import tapas.report

In [36]:
data = tapas.datasets.TabularDataset.read("Census_Microdata_Teaching_File")

FileNotFoundError: [Errno 2] No such file or directory: 'Census_Microdata_Teaching_File.csv'

In [17]:
generator = tapas.generators.Raw()

Nous définissons maintenant le modèle de menace. Les modèles de menace sont au cœur de TAPAS: elle englobe toutes les hypothèses faites à l'encontre de l'attaquant. Un modèle de menace a trois composantes: 1) ce que l'attaquant sait sur l'ensemble de données, 2) ce que l'attaquant sait sur le générateur, et 3) ce qu'il essaie d'en déduire.

Tout d'abord, nous définissons les connaissances de l'attaquant sur l'ensemble de données. Ici, nous supposons que l'attaquant a accès à un ensemble de données auxiliaires à partir de la même distribution, à partir de laquelle ils peuvent échantillonner des ensembles de données d'entraînement

Dans cet example, l'attaquant a accès à 50% du jeu de données total comme information auxiliaire. Ces informations seront utilisées pour produire des ensembles de données de formation. L'attaquant sait que l'ensemble de données réel contient 5000 échantillons. Cela reflète ainsi la connaissance de l'attaquant sur les données réelles.

Nous définissons ensuite ce que l'attaquant sait sur le générateur de données synthétiques. Il s'agirait généralement d'une connaissance de la boîte noire: l'attaquant est capable d'exécuter le modèle (exact) des ODD sur n'importe quel ensemble de données qu'il choisit mais ne peut observer que les paires (entrée, sortie) et non les paramètres internes. L'attaquant spécifie également la taille de l'ensemble de données de sortie (synthétique).

In [18]:
data_knowledge = tapas.threat_models.AuxiliaryDataKnowledge(
      data,
      auxiliary_split = 0.5,
      num_training_records = 5000
)

sdg_knowledge = tapas.threat_models.BlackBoxKnowledge(
    generator,
    num_synthetic_records = 5000,
)

NameError: name 'data' is not defined

Enfin, après avoir défini les connaissances de l’attaquant, nous précisons leur objectif – ce qu’il essaie de déduire. Nous nous concentrerons ici sur une attaque d'inférence de membres sur un registre cible. Nous sélectionnons ici (arbitrairement) le premier enregistrement dans l'ensemble de données.

Les trois premiers paramètres spécifient les composants de la connaissance de l'attaquant que nous avons définis ci-dessus, et l'enregistrement de la cible. Les deux derniers paramètres indiquent comment l'attaquant sera formé (par exemple, nous générons des paires pour former l'attaque). Pour plus de détails, veuillez consulter la documentation de l'API.

In [ ]:
threat_model = tapas.threat_models.TargetedMIA(
   attacker_knowledge_data = data_knowledge,
   target_record = data.get_records([0]),
   attacker_knowledge_generator = sdg_knowledge,
   generate_pairs = True,
   replace_target = True
)

Le modèle de menace représente ce qu'un attaquant peut faire. Nous pouvons maintenant définir un attaquant dont les capacités sont autorisées par le modèle de menace. Dans cet example, on instancie l'attaque GroundHog avec des paramètres standards (de Stadler et al., 2022), qui est fournie comme c'est le cas à l'intérieur tapas.

In [ ]:
attacker = tapas.attacks.GroundhogAttack()

L'attaque au sol est en fait une instanciation particulière d'une plus grande classe d'attaques. Il pourrait être rédigé de manière équivalente en utilisant les éléments suivants: TAPASclasses (se référer à la documentation pour plus de détails):

In [ ]:
from sklearn.ensemble import RandomForestClassifier

attacker = tapas.attacks.ShadowModellingAttack(
   tapas.attacks.FeatureBasedSetClassifier(
      tapas.attacks.NaiveSetFeature() + tapas.attacks.HistSetFeature() + tapas.attacks.CorrSetFeature(),
      RandomForestClassifier(n_estimators = 100)
   ),
   label = "Groundhog"
)

En TAPAS, les attaques doivent être formées pour un modèle de menace spécifique. Cette formation implique 1) des paramètres de réglage liés à l'ensemble de données (comme le nom des attributs et le nombre de catégories), et 2) l'entraînement de paramètres de modèle interne pour l'attaque. Dans la phase d'entraînement, l'attaquant voit un grand nombre d'ensembles de données « réels » générés en fonction des connaissances de l'attaquant et des ensembles de données synthétiques générés à partir de ces ensembles de données réels. Ces paires peuvent être utilisées pour, par exemple, former un classificateur pour inférer quelque chose sur l'ensemble de données réel à partir de l'ensemble de données synthétiques. Dans ce cas, le classificateur sera formé pour déterminer si un enregistrement spécifique figure dans l'ensemble de données réel. Cela se fait à l'aide de la .trainméthode, qui nécessite également de spécifier le nombre d'échantillons de formation (paires d'ensembles de données réelles et synthétiques):

In [ ]:
attacker.train(threat_model, num_samples = 1000)

Le TargetedMIAle modèle de menace est un modèle de menace TrainableThreatModelil définit une méthode pour générer des échantillons d'apprentissage (synthétique-dosset, target-in-real-dataset). C'est pourquoi le modèle de menace est passé pour former l'attaquant. Le num_samplesparamètre est le nombre de paires d'apprentissage générées par le modèle de menace pour former l'attaquant.

L'évaluation de l'attaque se fait dans le cadre de l'objet modèle de menace. En effet, conceptuellement, l'évaluation nécessite un accès à des informations que l'attaquant ne connaît pas (par exemple, les 50 % restants de l'ensemble de données). L'évaluation fonctionne de manière similaire à la formation en ce sens qu'un grand nombre d'ensembles de données « réels » et synthétiques sont générés, l'attaque est appliquée à chaque ensemble de données synthétiques et le taux de réussite de l'attaque est mesuré. Cela peut être fait avec le threat_model.testfonction:

In [ ]:
attack_summary = threat_model.test(attacker, num_samples = 100)

Ces produits a tapas.report.MIAttackSummaryobjet, qui contient les étiquettes des ensembles de données « réelles » d'apprentissage et l'étiquette (et le score) prédit par l'attaque. Vous pouvez soit utiliser ce résumé directement (il contient une gamme de statistiques agrégées) :

In [ ]:
metrics = attack_summary.get_metrics()
print("Results:\n", metrics.head())

In [ ]:
report = tapas.report.BinaryAIAttackReport(
   [attack_summary], metrics = ["accuracy", "privacy_gain", "auc"]
)
report.publish('groundhog-census')